# Renaming Abstracts

After examining around 5000 abstracts manually I found the following:

Empty abstracts - nothing we can do about that

Abstract containing only "None", "unk", "Not avail",  etc. --> define a min length for abstract, otherwise erase it and make an empty abstract

Abstract starting with "INTRODUCTION:" "what you need to know", "OBJECTIVE:", "OBJECTIVES", "Objective", "BACKGROUND:", "PURPOSE", "License:", "Abstract:", "Abstract: ", "ABSTRACT:" -->these words are predictable so we can simply delete them

Abstract starting with "&lt:p&gt:&", "10.1186/1471-25" --> delete the first work if it has too many numbers or non letters characters

Abstracts that repeat the title --> empty the abstract

Abstracts that finish with "..." --> remove the ...

Non alphanumerical characters --> not sure hoe to solve this

Chinese characters --> not sure how to solve this

**Overall need to do some code to better understand these abstracts** --> this is what this code is for


## Results

**empty Abstracts** are Np.NaN (less than 60 chararcters are also NaN, with a few exeptions).



## Still TODO (relatively simple)

Abstract starting with "INTRODUCTION:" "what you need to know", "OBJECTIVE:", "OBJECTIVES", "Objective", "BACKGROUND:", "PURPOSE", "License:", "Abstract:", "Abstract: ", "ABSTRACT:", "Summary" -->these words are predictable so we can simply delete them (I just want to see if there is any other words like this preceding the abstracts)

Abstract starting with "&lt:p&gt:&", "10.1186/1471-25" --> delete the first work if it has too many numbers or non letters characters

Abstract that talk about the availability and website etc. Want to delete this part from the abstract

In [1]:
from os.path import join as ospj
import os
import sys
import pandas as pd
import numpy as np
import sys
print(sys.version)
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import math

3.6.10 (default, Dec 19 2019, 23:04:32) 
[GCC 5.4.0 20160609]


In [2]:
import re

In [3]:
def clean_and_tokenize(text):
    """
    Cleaning a document with:
        - Lowercase        
        - Removing numbers with regular expressions
        - Removing punctuation with regular expressions
        - Removing other artifacts
    And separate the document into words by simply splitting at spaces
    Params:
        text (string): a sentence or a document
    Returns:
        tokens (list of strings): the list of tokens (word units) forming the document
    """        
    # Lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r"[0-9]+", "", text)
    # Remove punctuation
    REMOVE_PUNCT = re.compile("[.;:!\'?,\"()\[\]]")
    text = REMOVE_PUNCT.sub("", text)
    # Remove HTML artifacts specific to the corpus we're going to work with
    REPLACE_HTML = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    text = REPLACE_HTML.sub(" ", text)
    
    tokens = text.split()        
    return tokens

In [4]:
pd.set_option('display.max_rows', 25)

data_path = '/home/marchaln/CodeVsCovid19/data/metadata.csv'
data_path = '../data'

metadata_path = ospj(data_path, 'metadata.csv')
new_metadata_path = ospj(data_path, 'metadata_new_new.csv')

metadata_pd = pd.read_csv(metadata_path)  

In [5]:
#keep track of the number of changes we do
tot_changes = 0

In [6]:
#delete abstracts with less than 30 chararcters

df = metadata_pd[['title','abstract']]

for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    if len(str(one_abs)) < 30:
        df.at[i,'abstract']=np.NaN
        
        if not pd.isnull(one_abs) :
            tot_changes +=1 
            if one_abs != 'Unknown'and one_abs != '[Image: see text]':
                print(str(one_abs))
                
print('number of changes :', tot_changes )

None
Wash hands thoroughly.
Not Avail.
A Foresight study
While …
Abstract
Abstract No Abstract
[Figure: see text]
Set by Richard Smyth
number of changes : 351


In [7]:
# this is used to find the abstract of less than 60 chararcters not to delete

exceptions = ['Approvals, new indications, regulatory activities, and more',
             'Director general of the World Health Organization',
             'PR chief says they must learn how to present their stories',
             'Strategic risk communication for new and emerging risks',
             'Fast action will help the world contain the new coronavirus',
             'Sorting the week\'s supernovae from the absolute zeros',
             'Many drugs and vaccines are now being developed and tested']

tot_changes_prior = tot_changes 

for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    if len(str(one_abs)) < 60 and not pd.isnull(one_abs) :
        
        
        if str(one_abs) in exceptions:
            print('NOT DELETED')
            print(str(df['title'][i]))
            print(str(one_abs)+ '\n')
            
        else:
            print('DELETED')
            print(str(df['title'][i]))
            print(str(one_abs)+ '\n')    
            df.at[i,'abstract']=np.NaN 
            tot_changes += 1
            
print('number of changes :', tot_changes - tot_changes_prior )

DELETED
Resource Allocation during an Influenza Pandemic
Resource Allocation during an Influenza Pandemic

DELETED
Coronaviruses: Molecular and Cellular Biology
Coronaviruses: Molecular and Cellular Biology

DELETED
Isolation of Novel Adenovirus from Fruit Bat (Pteropus dasymallus yayeyamae)
Isolation of Novel Adenovirus from Fruit Bat

DELETED
Manual of Travel Medicine and Health
Manual of Travel Medicine and Health

DELETED
Ethical Alternatives to Experiments with Novel Potential Pandemic Pathogens
Please see later in the article for the Editors' Summary

NOT DELETED
Lee Jong-wook
Director general of the World Health Organization

NOT DELETED
Drug and Device News
Approvals, new indications, regulatory activities, and more

DELETED
Flu: A Medical Mystery
BBC Radio 4, 2 February at 8 pm Rating: ★★★

NOT DELETED
Public health doctors "hopeless" at using media
PR chief says they must learn how to present their stories

NOT DELETED
When even the 'best-laid' plans go wrong
Strategic risk c

In [8]:
#delete when the abstract is a duplicate of the title

tot_changes_prior = tot_changes 

for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    one_title = df['title'][i]
    diff = len(str(one_abs)) - len(str(one_title))
    diff = math.fabs(int(diff))
    
    if not pd.isnull(one_abs):
        #it is not always a pure duplicate but there is no more information than alredy in the title
        if (one_abs in one_title) or (one_title in one_abs) and diff < 200:
            if (one_abs[-3:] != '...')  :
                if (one_abs[-1:] != "\u2026") :
                    print('DELETE similar title and abstract:')
                    print('TITLE')
                    print(one_title)
                    print('ABSTRACT')
                    print(one_abs)
                    print('\n')
                    df.at[i,'abstract']=np.NaN 
                    tot_changes += 1
                else:
                    print('KEEP similar title and abstract:')
                    print('TITLE')
                    print(one_title)
                    print('ABSTRACT')
                    print(one_abs)
                    print('\n')
            else:
                print('KEEP similar title and abstract:')
                print('TITLE')
                print(one_title)
                print('ABSTRACT')
                print(one_abs)
                print('\n')


print('number of changes :', tot_changes - tot_changes_prior )

DELETE similar title and abstract:
TITLE
Stability of SARS-CoV-2 in different environmental conditions
ABSTRACT
Stability of SARS-CoV-2 in different environmental conditions.


DELETE similar title and abstract:
TITLE
Statistics-Based Predictions of Coronavirus Epidemic Spreading in Mainland China
ABSTRACT
Information about the open-access article 'Statistics-Based Predictions of Coronavirus Epidemic Spreading in Mainland China' in DOAJ. DOAJ is an online directory that indexes and provides access to quality open access, peer-reviewed journals.


DELETE similar title and abstract:
TITLE
Wuhan novel coronavirus 2019nCoV
ABSTRACT
Information about the open-access article 'Wuhan novel coronavirus 2019nCoV' in DOAJ. DOAJ is an online directory that indexes and provides access to quality open access, peer-reviewed journals.


KEEP similar title and abstract:
TITLE
Virtually Perfect? Telemedicine for Covid-19
ABSTRACT
Virtually Perfect? Telemedicine for Covid-19 Telemedicine’s payment and re

In [9]:
#find out the number of articles for which the abstract ends with "..." :

tot_changes_prior = tot_changes 

for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    one_title = df['title'][i]
    if not pd.isnull(one_abs):
        if one_abs[-3:] == '...' :
            print(one_abs)
            print('\n')
            df.at[i,'abstract']= one_abs[:-3]
            tot_changes += 1
        if one_abs[-1:] ==  "\u2026":
            print(one_abs)
            print('\n')
            df.at[i,'abstract']= one_abs[:-1]
            tot_changes += 1
            
print('number of changes :', tot_changes - tot_changes_prior )
            
#That turns out there are 20 of them but it is not a problem because the abstract 
# are long enough to uderstand what the paper talks about (sort of)
# however we must delete the ... because nlp might use this to think that these abstracts
# have something in common

Virtually Perfect? Telemedicine for Covid-19 Telemedicine’s payment and regulatory structures, licensing, credentialing, and implementation take time to work through, but health systems that have a...


Now calm down, we’re not about to start dolling out fashion tips for your facial fuzz. We’re talking about how having a hairy face could stop protective face masks from fitting properly.It can do. Derek Sandeman, the medical director of the University Hospital Southampton NHS Foundation Trust, has written to his clinical staff asking anyone working in high risk …


Healthcare staff on leave and those who have retired could be called “back to duty,” and non-urgent care could be delayed, as doctors are forced to prioritise dealing with covid-19, the UK government’s action plan lays out.The government will also implement a “distribution strategy for the UK’s stockpiles of key medicines and equipment (e.g. protective clothing),” the document said.However, the plan did not include details of

Infectious diseases are caused by infectious organisms. Typically, these are bacteria, …


The influenza viruses, which contain single-stranded RNA, are classified into 3 types, A, B, and C …


Bronchitis is characterized by bronchial inflammation that results in …


number of changes : 20


In [13]:
#Abstract starting with 
#a space or "/"
# "INTRODUCTION:" 
# "what you need to know", 
# "OBJECTIVE: ", "OBJECTIVE ", "OBJECTIVE. ", "Objective", "OBJECTIVE(S): ", "OBJECTIVES AND DESIGN: ", "OBJECTIVE--"
# "OBJECTIVES: ", "OBJECTIVES. ", "OBJECTIVES "
#  "BACKGROUND: ", "BACKGROUND ", "BACKGROUND"
# "PURPOSE: ", "PURPOSE OF REVIEW: ", "PURPOSE OF THE REVIEW: ", "PURPOSE AND EXPERIMENTAL DESIGN: ", "PURPOSE ", "Purpose. "
# "Abstract:", "Abstract: ", "ABSTRACT:",
#  "Summary",  "Summary ",  "Summary: ", "Summary / Abstract", 

# "motivation", "importance", "overview", "aim", " Rationale", "Objetivo: ", "Propósito: ",
# "S", "Context", "[Image: see text]", "-: "Conclusion", "Methods", "teaser", "issue"

# -->these words are predictable so we can simply delete them 
#(I just want to see if there is any other words like this preceding the abstracts)

INTRO_count = 0
what_you_need_to_know_count = 0
OBJECTIVE_count = 0
ABSTRACT_count = 0
BACKGROUND_count = 0
PURPOSE_count = 0
SUMMARY_count = 0
AIM_count = 0
S_count = 0
IMPORTANCE_count = 0
IMAGE_count = 0
TABLE_count = 0

def remove_beginning(df, i, keyword):
    
    count = 0
    length = len(keyword)
    
    one_abs = df['abstract'][i]
    while one_abs[0] == " " or one_abs[0] == "/":
        one_abs = one_abs[1:]
        df.at[i,'abstract']= one_abs
        
    if one_abs[:length+3].lower() == keyword + " / ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+3:]
    elif one_abs[:length+4].lower() == keyword + "s / ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+4:]
    elif one_abs[:length+1].lower() == keyword + " / ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+1:]
    elif one_abs[:length+4].lower() == keyword + "s / ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+4:]
    elif one_abs[:length+3].lower() == keyword + " : ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+3:]
    elif one_abs[:length+2].lower() == keyword + ": ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+2:]
    elif one_abs[:length+2].lower() == keyword + ". ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+2:]
    elif one_abs[:length+1].lower() == keyword + " ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+1:]
    # plurals
    elif one_abs[:length+4].lower() == keyword + "s : ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+4:]
    elif one_abs[:length+3].lower() == keyword + "s: ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+3:]
    elif one_abs[:length+2].lower() == keyword + ". ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+2:]
    elif one_abs[:length+2].lower() == keyword + "s ":    
        count += 1
        df.at[i,'abstract']= one_abs[length+2:]
    elif one_abs[:length+1].lower() == keyword + "s":    
        count += 1
        df.at[i,'abstract']= one_abs[length+1:]
    elif one_abs[:length+0].lower() == keyword:    
        count += 1
        df.at[i,'abstract']= one_abs[length+0]

    
    


for i, row in df.iterrows():
    one_abs = df['abstract'][i]
    one_title = df['title'][i]
    if not pd.isnull(one_abs):
        
        # delete space or / as first character
        while one_abs[0] == " " or one_abs[0] == "/":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
            
            
        # summary
        if one_abs[:9].lower() == "summary: ":    
            SUMMARY_count += 1
            df.at[i,'abstract']= one_abs[9:]
        elif one_abs[:10].lower() == "summary / ":    
            SUMMARY_count += 1
            df.at[i,'abstract']= one_abs[10:]
        elif one_abs[:8].lower() == "summary/":    
            SUMMARY_count += 1
            df.at[i,'abstract']= one_abs[8:]
        elif one_abs[:18].lower() == "summary objective ":    
            SUMMARY_count += 1
            df.at[i,'abstract']= one_abs[18:] 
        elif one_abs[:19].lower() == "summary background ":    
            SUMMARY_count += 1
            df.at[i,'abstract']= one_abs[19:]
        elif one_abs[:20].lower() == "summary background: ":    
            SUMMARY_count += 1
            df.at[i,'abstract']= one_abs[20:]
        elif one_abs[:8].lower() == "summary ":    
            SUMMARY_count += 1
            df.at[i,'abstract']= one_abs[8:]
        elif one_abs[:7].lower() == "summary":    
            SUMMARY_count += 1
            df.at[i,'abstract']= one_abs[7:]
            
        
        #INTRODUCTIOM
        one_abs = df['abstract'][i]
        while one_abs[0] == " " or one_abs[0] == "/":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
            
        if one_abs[:14].lower() == "introduction: ":
            INTRO_count += 1
            df.at[i,'abstract']= one_abs[14:]
        elif one_abs[:14].lower() == "introduction. ":
            INTRO_count += 1
            df.at[i,'abstract']= one_abs[14:]
        elif one_abs[:13].lower() == "introduction ":
            INTRO_count += 1
            df.at[i,'abstract']= one_abs[13:]
        elif one_abs[:12].lower() == "introduction":
            INTRO_count += 1
            df.at[i,'abstract']= one_abs[12:]
                
            
        # "What you need to know"    
        elif one_abs[:21] == "What you need to know":
            what_you_need_to_know_count += 1
            df.at[i,'abstract']= one_abs[21:]
        
        
        one_abs = df['abstract'][i]
        while one_abs[0] == " " or one_abs[0] == "/":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
        
        #Abstract
        if one_abs[:10].lower() == "abstract: ":    
            ABSTRACT_count += 1
            df.at[i,'abstract']= one_abs[10:]
        elif one_abs[:11].lower() == "abstracts: ":    
            ABSTRACT_count += 1
            df.at[i,'abstract']= one_abs[11:]
        elif one_abs[:9].lower() == "abstracts":    
            ABSTRACT_count += 1
            df.at[i,'abstract']= one_abs[9:]
        elif one_abs[:8].lower() == "abstract":    
            ABSTRACT_count += 1
            df.at[i,'abstract']= one_abs[8:]
            
        
            
        #Background
        
        while one_abs[0] == " " or one_abs[0] == "/":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
        if one_abs[:13].lower() == "background : ":    
            BACKGROUND_count += 1
            
            df.at[i,'abstract']= one_abs[13:]
        if one_abs[:12].lower() == "background: ":    
            BACKGROUND_count += 1
            df.at[i,'abstract']= one_abs[12:]
        elif one_abs[:11].lower() == "background ":    
            BACKGROUND_count += 1
            df.at[i,'abstract']= one_abs[11:]
        elif one_abs[:13].lower() == "backgrounds: ":    
            BACKGROUND_count += 1
            df.at[i,'abstract']= one_abs[13:] 
        elif one_abs[:12].lower() == "backgrounds ":    
            BACKGROUND_count += 1
            df.at[i,'abstract']= one_abs[12:] 
        elif one_abs[:10].lower() == "background":    
            BACKGROUND_count += 1
            df.at[i,'abstract']= one_abs[10:]  
        elif one_abs[:11].lower() == "backgound: ":  #spelling mistake  
            BACKGROUND_count += 1
            df.at[i,'abstract']= one_abs[11:]
        elif one_abs[:11].lower() == "backgroud: ":  #spelling mistake  
            BACKGROUND_count += 1
            df.at[i,'abstract']= one_abs[11:]
            
        #PURPOSE
        one_abs = df['abstract'][i]
        while one_abs[0] == " " or one_abs[0] == "/":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
        if one_abs[:33].lower() == "purpose and experimental design: ":    
            PURPOSE_count += 1
            df.at[i,'abstract']= one_abs[33:]
        elif one_abs[:23].lower() == "purpose of the review: ":    
            PURPOSE_count += 1
            df.at[i,'abstract']= one_abs[23:]
        elif one_abs[:19].lower() == "purpose of review: ":    
            PURPOSE_count += 1
            df.at[i,'abstract']= one_abs[19:]
        elif one_abs[:9].lower() == "purpose. ":    
            PURPOSE_count += 1
            df.at[i,'abstract']= one_abs[9:]
        elif one_abs[:9].lower() == "purpose: ":    
            PURPOSE_count += 1
            df.at[i,'abstract']= one_abs[9:]
        elif one_abs[:8].lower() == "purpose ":    
            PURPOSE_count += 1
            df.at[i,'abstract']= one_abs[8:]
        elif one_abs[:8].lower() == "urpose: ":    
            PURPOSE_count += 1
            df.at[i,'abstract']= one_abs[8:]
            
        #aim:
        one_abs = df['abstract'][i]
        while one_abs[0] == " " or one_abs[0] == "/":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
            
        if one_abs[:6].lower() == "aims: ":    
            AIM_count += 1
            df.at[i,'abstract']= one_abs[6:]
        elif one_abs[:5].lower() == "aims ":    
            AIM_count += 1
            df.at[i,'abstract']= one_abs[5:]
        elif one_abs[:6].lower() == "aims--":    
            AIM_count += 1
            df.at[i,'abstract']= one_abs[6:]
        elif one_abs[:4].lower() == "aims":    
            AIM_count += 1
            df.at[i,'abstract']= one_abs[4:]
        elif one_abs[:5].lower() == "aim: ":    
            AIM_count += 1
            df.at[i,'abstract']= one_abs[5:]
        elif one_abs[:5].lower() == "aim. ":    
            AIM_count += 1
            df.at[i,'abstract']= one_abs[5:]
        elif one_abs[:4].lower() == "aim ":    
            AIM_count += 1
            df.at[i,'abstract']= one_abs[4:]
            
              
        #importance: IMPORTANCE OF THE FIELD:
        elif one_abs[:11].lower() == "importances":    
            IMPORTANCE_count += 1
            print(one_abs + '\n')            
        elif one_abs[:25].lower() == "importance of the field: ":    
            IMPORTANCE_count += 1
            df.at[i,'abstract']= one_abs[25:]
        elif one_abs[:12].lower() == "importance: ":    
            IMPORTANCE_count += 1
            df.at[i,'abstract']= one_abs[12:]
        elif one_abs[:11].lower() == "importance ":    
            IMPORTANCE_count += 1
            df.at[i,'abstract']= one_abs[11:]
            
        
        #OBJECTIVE (again)
        one_abs = df['abstract'][i]
        while one_abs[0] == " " or one_abs[0] == "/":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
            
        if one_abs[:23].lower() == "objectives and design: ":
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[23:]
            
        elif one_abs[:14].lower() == "objective(s): ":
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[14:]
            
        elif one_abs[:11].lower() == "objective--":
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[11:]
            
        elif one_abs[:11].lower() == "objectives—":
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[11:]
            
        elif one_abs[:12].lower() == "objectives. ":     
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[12:]
            
        elif one_abs[:12].lower() == "objectives: ":
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[12:]
            
        elif one_abs[:11].lower() == "objectives ":
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[11:]
            
        elif one_abs[:11].lower() == "objective. ":    
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[11:]
            
        elif one_abs[:11].lower() == "objective: ":    
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[11:]
            
        elif one_abs[:10].lower() == "objective ":    
            OBJECTIVE_count += 1
            df.at[i,'abstract']= one_abs[10:]
            
        #INTRODUCTION (again)
        one_abs = df['abstract'][i]
        while one_abs[0] == " " or one_abs[0] == "/":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
            
        if one_abs[:14].lower() == "introduction: ":
            INTRO_count += 1
            df.at[i,'abstract']= one_abs[14:]
        elif one_abs[:14].lower() == "introduction. ":
            INTRO_count += 1
            df.at[i,'abstract']= one_abs[14:]
        elif one_abs[:13].lower() == "introduction ":
            INTRO_count += 1
            df.at[i,'abstract']= one_abs[13:]
        elif one_abs[:12].lower() == "introduction":
            INTRO_count += 1
            df.at[i,'abstract']= one_abs[12:]
            
            
        
        # [Image: see text]
        if one_abs[:18].lower() == "[image: see text] ":    
            IMAGE_count += 1
            df.at[i,'abstract']= one_abs[18:]  
            
        
        # [Table: see text] 
        if one_abs[:18].lower() == "[table: see text] ":    
            TABLE_count += 1
            df.at[i,'abstract']= one_abs[18:]  
            
        #S:
        one_abs = df['abstract'][i]
        if one_abs[:3].lower() == "s: ":    
            S_count += 1
            df.at[i,'abstract']= one_abs[3:]
            
        
        one_abs = df['abstract'][i]
        while one_abs[0] == " " or one_abs[0] == "/" or one_abs[0] == "-" or one_abs[0] == ":":
            one_abs = one_abs[1:]
            df.at[i,'abstract']= one_abs
        
        #test the rest
        one_abs = df['abstract'][i]
        if one_abs.split()[0][-1] == ":":    
            #pass
            print(one_abs + '\n')
            #SUMMARY_count += 1
            #df.at[i,'abstract']= one_abs[7:]
        

            
print('Number of \'INTRODUCTION: \' :', INTRO_count)
print('Number of \'what you need to know\' :', what_you_need_to_know_count )
print('Number of \'OBJECTIVE\' :', OBJECTIVE_count)
print('Number of \'ABSTRACT\'  :', ABSTRACT_count)
print('Number of \'BACKGROUND\'  :', BACKGROUND_count)
print('Number of \'PURPOSE\'  :', PURPOSE_count)
print('Number of \'SUMMARY\'  :', SUMMARY_count)
print('Number of \'AIM\'  :', AIM_count)
print('Number of \'S\'  :', S_count) 
print('Number of \'IMPORTANCE\'  :', IMPORTANCE_count) 
print('Number of \'IMAGE\'  :', IMAGE_count) 
print('Number of \'TABLE\'  :', TABLE_count) 


tot_changes += INTRO_count + \
                what_you_need_to_know_count +\
                OBJECTIVE_count +\
                ABSTRACT_count +\
                BACKGROUND_count +\
                PURPOSE_count +\
                SUMMARY_count +\
                AIM_count +\
                S_count +\
                IMPORTANCE_count +\
                IMAGE_count+\
                TABLE_count

Motivation: Wuhan pneumonia is an acute infectious disease caused by the 2019 novel coronavirus (COVID-19). It is being treated as a Class A infectious disease though it was classified as Class B according to the Infectious Disease Prevention Act of China. Accurate estimation of the incubation period of the coronavirus is essential to the prevention and control. However, it remains unclear about its exact incubation period though it is believed that symptoms of COVID-19 can appear in as few as 2 days or as long as 14 or even more after exposure. The accurate incubation period calculation requires original chain-of-infection data that may not be fully available in the Wuhan regions. In this study, we aim to accurately calculate the incubation period of COVID-19 by taking advantage of the chain-of-infection data, which is well-documented and epidemiologically informative, outside the Wuhan regions. Methods: We acquired and collected officially reported COVID-19 data from 10 regions in Ch

CONTEXT: There have been few longitudinal studies of cytokine production in neonatally acquired HIV-1 infection and none in Asian or Chinese children. OBJECTIVE: To determine whether monitoring cytokine production could contribute to the better management of pediatric patients with HIV-1 infection. SETTING: Clinical Immunology Laboratory and Pediatrics Department, University Hospital, Hong Kong. PATIENTS: Ten Asian and 2 Eurasian children infected with HIV-1 by mother-to-child transmission were followed for up to 5 years while on treatment with highly active antiretroviral therapy (HAART). MAIN OUTCOME MEASURES: Numbers of unstimulated and mitogen-activated cytokine-secreting cells (IFN-gamma, interleukin [IL]-2, IL-4, IL-6, IL-10, IL-12, and TNF-alpha) were measured by ELISPOT assay at frequent intervals, and correlations were sought with CD4+ and CD8+ cell counts and viral loads. RESULTS: Mitogen-stimulated IL-2-secreting cells were directly associated with recovery of CD4+ cells. Co

Motivation: The study of RNA virus populations is a challenging task. Each population of RNA virus is composed of a collection of different, yet related genomes often referred to as mutant spectra or quasispecies. Virologists using deep sequencing technologies face major obstacles when studying virus population dynamics, both experimentally and in natural settings due to the relatively high error rates of these technologies and the lack of high performance pipelines. In order to overcome these hurdles we developed a computational pipeline, termed ViVan (Viral Variance Analysis). ViVan is a complete pipeline facilitating the identification, characterization and comparison of sequence variance in deep sequenced virus populations. Results: Applying ViVan on deep sequenced data obtained from samples that were previously characterized by more classical approaches, we uncovered novel and potentially crucial aspects of virus populations. With our experimental work, we illustrate how ViVan can

RATIONALE: Hemophagocytic lymphohistiocytosis (HLH) is a syndrome that is characterized by an inappropriate hyperinflammatory immune response – primary, as a consequence of a genetic defect of NK cells and cytotoxic T lymphocytes or – secondary, in the progression of infections, rheumatic or autoimmune diseases, malignancies or metabolic diseases. PATIENT CONCERNS: We present the case of a secondary HLH due to Streptococcus pneumoniae infection in a splenectomised patient for spherocytosis, a 37-year-old patient who was splenectomised in childhood for spherocytosis, without immuneprophylaxis induced by antipneumococcal vaccine. OUTCOMES: He developed a severe pneumococcal sepsis associated with secondary HLH, with unfavorable outcome and death. LESSONS: To our knowledge, just 2 similar cases had been published in the literature, none in which the secondary HLH was the consequence of an invasive pneumococcal infection in a splenectomized patient for spherocytosis, and the association of

MOTIVATION: To understand protein structure, folding and function fully and to design proteins de novo reliably, we must learn from natural protein structures that have been characterized experimentally. The number of protein structures available is large and growing exponentially, which makes this task challenging. Indeed, computational resources are becoming increasingly important for classifying and analyzing this resource. Here, we use tools from graph theory to define an Atlas classification scheme for automatically categorizing certain protein substructures. RESULTS: Focusing on the α-helical coiled coils, which are ubiquitous protein-structure and protein–protein interaction motifs, we present a suite of computational resources designed for analyzing these assemblies. iSOCKET enables interactive analysis of side-chain packing within proteins to identify coiled coils automatically and with considerable user control. Applying a graph theory-based Atlas classification scheme to str

...

Rationale: An activated vasoconstrictive, proliferative, and fibrotic axis of the renin angiotensin system (angiotensin-converting enzyme [ACE]/angiotensin [Ang]II/AngII type 1 receptor) has been implicated in the pathophysiology of pulmonary fibrosis (PF) and pulmonary hypertension (PH). The recent discovery of a counterregulatory axis of the renin angiotensin system composed of ACE2/Ang-(1–7)/Mas has led us to examine the role of this vasoprotective axis on such disorders. Objectives: We hypothesized that Ang-(1–7) treatment would exert protective effects against PF and PH. Methods: Lentiviral packaged Ang-(1–7) fusion gene or ACE2 cDNA was intratracheally administered into the lungs of male Sprague Dawley rats. Two weeks after gene transfer, animals received bleomycin (2.5 mg/kg). In a subsequent study, animals were administered monocrotaline (MCT, 50 mg/kg). Measurements and Main Results: In the PF study, bleomycin administration resulted in a significant increase in right ventricu

PRESENTATION: Patient is a 6-year-old male with CF, MRSA colonization, and pancreatic insufficiency that presented with worsening ppFEV1 and systemic symptoms despite multiple interventions. BAL grew NTM, Stenotrophomonas maltophilia, and Inquilinus limosus, a rare organism found in patients with CF. COURSE: I. limosus treatment was deferred. Despite treatment of other pathogens, symptoms worsened. I. limosus was targeted with meropenem, amikacin, and ciprofloxacin along with clindamycin for MRSA colonization. Within weeks, symptoms had resolved with ppFEV1 improvement. DISCUSSION: This case discusses the importance of a rare organism in the CF population. Targeting I.limosus was key to recovery, revealing its potential pathogenicity.

Setting: The University of Washington Health Sciences Libraries and Information Center BioCommons serves the bioinformatics needs of researchers at the university and in the vibrant for-profit and not-for-profit biomedical research sector in the Washingt

Rationale: Acute lung dysfunction of noninfectious etiology, known as idiopathic pneumonia syndrome (IPS), is a severe complication following hematopoietic stem cell transplantation (HSCT). Several mouse models have been recently developed to determine the underlying causes of IPS. A cohesive interpretation of experimental data and their relationship to the findings of clinical research studies in humans is needed to better understand the basis for current and future clinical trials for the prevention/treatment of IPS. Objectives: Our goal was to perform a comprehensive review of the preclinical (i.e., murine models) and clinical research on IPS. Methods: An ATS committee performed PubMed and OVID searches for published, peer-reviewed articles using the keywords “idiopathic pneumonia syndrome” or “lung injury” or “pulmonary complications” AND “bone marrow transplant” or “hematopoietic stem cell transplant.” No specific inclusion or exclusion criteria were determined a priori for this r

CONTEXT: Respiratory infections are prevalent in the elderly, resulting in increased morbidity, mortality, and utilization of health care services. Vitamin E supplementation has been shown to improve immune response in the elderly. However, the clinical importance of these findings has not been determined. OBJECTIVE: To investigate the effect of 1-year vitamin E supplementation on respiratory infections in elderly nursing home residents DESIGN: A randomized, double-blind, placebo-controlled trial conducted from April 1998 to August 2001 SETTING: 33 long-term care facilities in the Boston, Massachusetts area PARTICIPANTS: 617 subjects ≥65 years old, who met the study’s eligibility criteria were enrolled, 73% of whom completed the study. The follow-up time (mean ± SD) was 317±104 and 321±97 days, E and placebo respectively, for all subjects enrolled in the study. INTERVENTION: A daily vitamin E (200 IU) or placebo capsule; all subjects received a capsule containing 1/2 the Recommended Da

Mediawatch: Lethal virus outbreaks may help sell newspapers but Bernard Dixon finds that the early response from journalists to the severe acute respiratory syndrome (SARS) has been more informed than in many previous scares with diseases resulting from other dangerous viruses.

Teaser: Our review found the average R0 for 2019-nCoV to be 3.28, which exceeds WHO estimates of 1.4 to 2.5.Here we review the basic reproduction number (R0) of the 2019-nCoV virus. R0 is an indication of the transmissibility of a virus, representing the average number of new infections generated by an infectious person in a totally naïve population. For R0 greater than one the number infected is likely to increase, and for R0 less than one transmission is likely to die out. The basic reproduction number is a central concept in infectious disease epidemiology, indicating the risk of an infectious agent with respect to epidemic spread.

Number of 'INTRODUCTION: ' : 0
Number of 'what you need to know' : 0
Number 

In [11]:
print('Total number of changes :', tot_changes)

Total number of changes : 17564


In [12]:
#save the new dataframe
if os.path.isfile(new_metadata_path):
    sys.exit('ERROR, a new dataset is already saved')
else:
    df.to_csv(new_metadata_path)

SystemExit: ERROR, a new dataset is already saved

/home/marchaln/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
